In [65]:
from datetime import timedelta
from datetime import datetime
import pandas as pd
from datetime import timezone
import numpy as np

In [66]:
start_date = datetime.fromisoformat("2018-01-01 00:00").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat("2018-12-31 23:59").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [67]:
apple_news = pd.read_pickle("pkls/apple_news_data.pkl", compression='infer')
amazon_news = pd.read_pickle("pkls/amazon_news_data.pkl", compression='infer')
apple_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')
amazon_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')

In [68]:
apple_news

,published_time,article_content,sentiments
16174,1.514832e+09,facebook \nciti analysts say that there is a 4...,0
15612,1.514855e+09,"quote: in this context avxl is a new apple, im...",1
15928,1.514862e+09,2 warren buffett stocks to consider buying now...,0
12200,1.514862e+09,2 warren buffett stocks to consider buying now...,0
14085,1.514862e+09,2 warren buffett stocks to consider buying now...,0
...,...,...,...
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ...",-1
61168,1.546322e+09,big trends: gartner iaas mq is down to 6 compa...,0
41,1.546326e+09,- fundamental analysis - technology is apple s...,0
60869,1.546332e+09,"new york, oct. 23, 2018 (globe newswire) -- in...",0


In [69]:
amazon_news

,published_time,article_content,sentiments
11106,1.514876e+09,"by ciara linnane and tomi kilgore, marketwatch...",0
11837,1.514876e+09,by ryan vlastelica\nto simply match the market...,0
15252,1.514876e+09,what happened shares of many optical networkin...,0
15737,1.514876e+09,by nigam arora\nthe practical way to take adva...,0
13476,1.514876e+09,-michael ashbaugh;\ncompany symbol date profil...,0
...,...,...,...
1348,1.546281e+09,the biggest benefit from living are the lesson...,0
2553,1.546290e+09,apple stock price\napple (nasdaq: aapl ) was v...,0
849,1.546306e+09,none of us has a crystal ball that shows us cl...,0
56164,1.546321e+09,"let's talk about apple, inc. (nasdaq: aapl ). ...",0


In [70]:
apple_chart["articles"] = [[] for i in range(len(apple_chart))]
apple_chart['sentiments'] = 0

In [71]:
amazon_chart

,timestamp,trends
2486,1.514925e+09,1
2487,1.514939e+09,1
2488,1.514954e+09,-1
2489,1.515011e+09,1
2490,1.515026e+09,-1
...,...,...
3180,1.545971e+09,-1
3181,1.546029e+09,-1
3182,1.546043e+09,-1
3183,1.546288e+09,1


In [80]:
index = 0
row = []
minutes = 240
try:
    for index, row in apple_chart.iterrows():
        time = row['timestamp']
        low_end = (datetime.utcfromtimestamp(row['timestamp']) - timedelta(minutes=minutes)).replace(tzinfo=timezone.utc).timestamp()
        articles = []
        sentiment = 0
        for news_index, news_row in apple_news.iterrows():
            if news_row['published_time'] > time:
                break
            if low_end <= news_row['published_time'] <= time:
                sentiment += news_row['sentiments']
                articles.append(news_row['article_content'])
        apple_chart.at[index, 'articles'] = articles
        apple_chart.at[index, 'sentiments'] = sentiment
except Exception as e:
    print(e)
    print(row, index)

In [81]:
apple_chart

,timestamp,trends,articles,sentiments
2486,1.514925e+09,1,"[thinking about investing in apple, alibaba, n...",-1
2487,1.514939e+09,1,[login search pre-market most active for jan 2...,-13
2488,1.514954e+09,-1,[steven mnuchin: taxes will go up for the rich...,-2
2489,1.515011e+09,1,[the stock market's animal spirits have been t...,0
2490,1.515026e+09,-1,[with reports of a potential bug in intel corp...,0
...,...,...,...,...
3180,1.545971e+09,-1,[source: apple.com\napple ( aapl ) has been on...,-5
3181,1.546029e+09,-1,[shutterstock photo top tech stocks\nmsft -0.9...,-3
3182,1.546043e+09,-1,[apple and netflix could be your best faang be...,-4
3183,1.546288e+09,1,[shutterstock photo top tech stocks\nmsft +0.7...,0


In [84]:
len(apple_chart['articles'][3182])

17

In [85]:
trends = apple_chart['trends'].to_numpy()
sentiments = apple_chart['sentiments'].to_numpy()
print(np.cov(trends, sentiments))

[[ 1.00034843  0.04577149]
 [ 0.04577149 10.96762055]]
